In [ ]:
from collections import deque
import cv2
import numpy as np

# Giả sử các biến này đã được định nghĩa ở nơi khác
MoBiLSTM_model = ...
SEQUENCE_LENGTH = ...
IMAGE_HEIGHT, IMAGE_WIDTH = ..., ...
CLASSES_LIST = [...]

def predict_rtsp_stream(rtsp_url, SEQUENCE_LENGTH):
    """
    Đọc và xử lý luồng RTSP, sau đó hiển thị kết quả.
    """
    
    # Kết nối tới luồng RTSP.
    video_reader = cv2.VideoCapture(rtsp_url)
    if not video_reader.isOpened():
        print(f"Lỗi: Không thể mở luồng RTSP tại {rtsp_url}")
        return

    # Khởi tạo hàng đợi để lưu trữ các frame.
    frames_queue = deque(maxlen=SEQUENCE_LENGTH)

    # Biến lưu trữ tên lớp dự đoán.
    predicted_class_name = ''

    while True: # Vòng lặp vô hạn để xử lý luồng liên tục
        
        ok, frame = video_reader.read()
        
        if not ok:
            print("Lỗi đọc frame hoặc luồng kết thúc. Đang thử kết nối lại...")
            video_reader.release()
            video_reader = cv2.VideoCapture(rtsp_url) # Cố gắng kết nối lại
            continue

        # Tiền xử lý frame.
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        normalized_frame = resized_frame / 255.0
        frames_queue.append(normalized_frame)

        # Chỉ dự đoán khi hàng đợi có đủ số frame.
        if len(frames_queue) == SEQUENCE_LENGTH:
            # Chuyển hàng đợi thành mảng numpy và dự đoán.
            input_data = np.expand_dims(frames_queue, axis=0)
            predicted_labels_probabilities = MoBiLSTM_model.predict(input_data)[0]
            
            # Lấy lớp có xác suất cao nhất.
            predicted_label = np.argmax(predicted_labels_probabilities)
            predicted_class_name = CLASSES_LIST[predicted_label]

        # Ghi tên lớp dự đoán lên frame gốc.
        if predicted_class_name == "Violence":
            cv2.putText(frame, predicted_class_name, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 0, 255), 4)
        else:
            cv2.putText(frame, predicted_class_name, (20, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 4)
            
        # Hiển thị frame.
        cv2.imshow('RTSP Stream Inference', frame)

        # Nhấn 'q' để thoát.
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
    # Dọn dẹp.
    video_reader.release()
    cv2.destroyAllWindows()


In [9]:
IMAGE_HEIGHT , IMAGE_WIDTH = 64, 64
 
SEQUENCE_LENGTH = 16
 
CLASSES_LIST = ["NonViolence","Violence"]

In [10]:
from tensorflow.keras.applications import MobileNet

mobilenet = MobileNet(
    input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3),
    include_top=False,
    weights='imagenet'
)

In [11]:
import os
import shutil
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow 
import keras
from collections import deque
import matplotlib.pyplot as plt

%matplotlib inline
 
# from sklearn.model_selection import train_test_split
 
from keras.layers import *
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

def create_model():
 
    model = Sequential()

    ########################################################################################################################
    
    #Specifying Input to match features shape
    model.add(Input(shape = (SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3)))
    
    # Passing mobilenet in the TimeDistributed layer to handle the sequence
    model.add(TimeDistributed(mobilenet))
    
    model.add(Dropout(0.25))
                                    
    model.add(TimeDistributed(Flatten()))

    
    lstm_fw = LSTM(units=32)
    lstm_bw = LSTM(units=32, go_backwards = True)  

    model.add(Bidirectional(lstm_fw, backward_layer = lstm_bw))
    
    model.add(Dropout(0.25))

    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(64,activation='relu'))
    model.add(Dropout(0.25))

    model.add(Dense(32,activation='relu'))
    model.add(Dropout(0.25))
    
    
    model.add(Dense(len(CLASSES_LIST), activation = 'softmax'))
 
    ########################################################################################################################
 
    model.summary()
    
    return model

In [15]:
model_b = create_model()
model_b.summary() # In tóm tắt kiến trúc của mô hình hiện tại

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_4 (TimeDist (None, 16, 2, 2, 1024)    3228864   
_________________________________________________________________
dropout_12 (Dropout)         (None, 16, 2, 2, 1024)    0         
_________________________________________________________________
time_distributed_5 (TimeDist (None, 16, 4096)          0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 64)                1057024   
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 256)               16640     
_________________________________________________________________
dropout_14 (Dropout)         (None, 256)              

In [12]:
from tensorflow.keras.models import load_model

In [16]:
model = load_model('trained_model.h5')

In [17]:
import tf2onnx
import tensorflow as tf


# Convert
spec = (tf.TensorSpec((None, SEQUENCE_LENGTH, IMAGE_HEIGHT, IMAGE_WIDTH, 3), tf.float32, name="input"),)
output_path = "model.onnx"
model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)

In [1]:
import onnx
from onnx2pytorch import ConvertModel

# Load ONNX
onnx_model = onnx.load("./model.onnx")

# Chuyển thành PyTorch, onnx2pytorch có hỗ trợ LSTM/GRU/RNN
pytorch_model = ConvertModel(onnx_model)

# Kiểm tra
print(pytorch_model)

# Lưu lại
import torch
torch.save(pytorch_model, "./pytorch_model.pth")


c:\Users\minh.nhat\AppData\Local\anaconda3\envs\natmin\Lib\site-packages\onnx2pytorch\convert\layer.py:30: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:212.)
  layer.weight.data = torch.from_numpy(numpy_helper.to_array(weight))


NotImplementedError: Extraction of attribute type 8 not implemented.